# Hello Mozfest!

This notebook shows how to get set up with the Overscripted dataset. It covers working on a sample of the data in [Pandas](http://pandas.pydata.org/), and processing the full data in [Spark](https://spark.apache.org/docs/latest/api/python/pyspark.html).

The [analysis README](https://github.com/mozilla/Overscripted-Data-Analysis-Challenge/tree/master/analyses) describes how to set up the environment with the necessary dependencies, including the Jupyter notebook. Specific packages are listed in the [environment.yaml](https://github.com/mozilla/Overscripted-Data-Analysis-Challenge/blob/master/analyses/environment.yaml).

In [1]:
import pandas as pd
import os.path

In [2]:
## Don't limit pandas display.
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", -1)

Specify the base dir where you have saved the dataset.

In [3]:
DATA_DIR = "../data/"  ## Change me

We are primarily interested in the `value` column, representing the value returned from each API call. We retain it and a few other relevant columns for analysis.

In [4]:
COLUMNS = [
    "location",  ## The URL of the page that was crawled
    "script_url",  ## The script where the call was made
    "symbol",  ## The JS API symbol
    "value",  ## The value returned
    "value_1000",  ## The value truncated to 1000 chars
    "value_len"  ## The number of chars in the full value string
]

## Working with a sample in Pandas

The sample dataset described in the [README](https://github.com/mozilla/Overscripted-Data-Analysis-Challenge) is small enough that we can load it into memory and explore it using Pandas.

Download and unzip it into the data dir. Then load it as a Pandas DataFrame, restricted to the columns of interest.

In [5]:
SAMPLE_DATA_PARQUET = os.path.join(DATA_DIR,
                                   "sample",
                                   "part-00000-34d9b361-ea79-42eb-82ee-9c9f9259c339-c000.snappy.parquet")

In [6]:
df = pd.read_parquet(SAMPLE_DATA_PARQUET,
                     engine="pyarrow",
                     columns=COLUMNS)

In [7]:
df.head()

,location,script_url,symbol,value,value_1000,value_len
0,https://staticxx.facebook.com/connect/xd_arbiter/r/lY4eZXm_YWu.js?version=42#channel=f30ef17b61f384&origin=http%3A%2F%2Fwww.ubitennis.com,https://staticxx.facebook.com/connect/xd_arbiter/r/lY4eZXm_YWu.js?version=42#channel=f30ef17b61f384&origin=http%3A%2F%2Fwww.ubitennis.com,window.name,fb_xdm_frame_https,fb_xdm_frame_https,18
1,https://staticxx.facebook.com/connect/xd_arbiter/r/lY4eZXm_YWu.js?version=42#channel=f30ef17b61f384&origin=http%3A%2F%2Fwww.ubitennis.com,https://staticxx.facebook.com/connect/xd_arbiter/r/lY4eZXm_YWu.js?version=42#channel=f30ef17b61f384&origin=http%3A%2F%2Fwww.ubitennis.com,window.name,fb_xdm_frame_https,fb_xdm_frame_https,18
2,https://staticxx.facebook.com/connect/xd_arbiter/r/lY4eZXm_YWu.js?version=42#channel=f30ef17b61f384&origin=http%3A%2F%2Fwww.ubitennis.com,https://staticxx.facebook.com/connect/xd_arbiter/r/lY4eZXm_YWu.js?version=42#channel=f30ef17b61f384&origin=http%3A%2F%2Fwww.ubitennis.com,window.document.cookie,,,0
3,https://staticxx.facebook.com/connect/xd_arbiter/r/lY4eZXm_YWu.js?version=42#channel=f30ef17b61f384&origin=http%3A%2F%2Fwww.ubitennis.com,https://staticxx.facebook.com/connect/xd_arbiter/r/lY4eZXm_YWu.js?version=42#channel=f30ef17b61f384&origin=http%3A%2F%2Fwww.ubitennis.com,window.navigator.userAgent,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,68
4,https://cas.us.criteo.com/delivery/r/afr.php?did=5a34c73ff17390f0eaeb591979874b00&z=WjTHPwAHZYkKT7BIAAcskrRh4Qozh3b-c-mtZg&u=%7C7J5NcLNwKWZvhHazrdQ0r3pEybQM2VrhNSue519M%2FnU%3D%7C&c1=M5BADJe1UR3zJ2HNju9b10FggySKKMK0AoYTtPDcqDnSIQIZUQPlDupK--OP2eR-eNGQ46cgN3mwCl5UMg4IstlvomsUbHEHUzImPBAbL0KpTFeMsdEkBo28MAQVY_79HvMen3pU9pjoRxbnxk_AxatU3fdvCPtFY7Wzui5q962zi71J5i_HHNmYi7XbHxLl1v3NLOEqWiI-3QfHE1byzwOhuyge44QAJfUpukDSr4X723xUoquihjIy6b6D_yU9AsLHIIxKQk64_ES4G8moUw5dbt7SG3KWRhyjzAZW5acfRwaX8v33UzaCSZKj4O0XffzJaDiMmsprtAOP0J4xHPtfZqvurKt_x3z5y83mK1o&ct0=https://adclick.g.doubleclick.net/aclk%3Fsa%3Dl%26ai%3DCcqaRP8c0WonLHcjgvgKS2ZyADO7lmPBNsu23nZ0BwI23ARABIABgyQaCARdjYS1wdWItNTc4NzU5MjQ4Mzc2Njc2MKABrN3-6APIAQngAgCoAwGqBMUBT9DGnU9Xf5zpWjsp7PXxVDLu7mhvsOzx8jjeTb-wk_FUQNpBqVd4QxwydKBkX31VemFtAuP1QMeGjoHagpA44JfU11OU46ZLmBKcADPeCDg8kDPJvowA7EbbZ6gvml2aRO7nKo1LHNbLoGTBvP6gmhnbhVqThagbrECDM6qxbcRiiWobTKajDG8KeWma5flmrMZiQe5Lu3cyX_WMmu36IIP2lojiMZaZvgiE_ncYb24UZCKxrORb0gO54t1XHQFwzDnBRPXgBAGABufvkeKYhIzL9gGgBiGoB6a-G9gHANIIBQiAYRAB8ggbYWR4LXN1YnN5bi0wOTI1MDI4NTk2NjIxNjE3%26num%3D1%26sig%3DAOD64_1ibrZmc1pVLttz4doqoDmSHOvwXg%26client%3Dca-pub-5787592483766760%26adurl%3D,https://ajax.googleapis.com/ajax/libs/webfont/1.6.26/webfont.js,window.navigator.userAgent,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,68


Restrict to operations that returned a value.

In [8]:
df = df[df.value_len > 0]

How many rows with values are there in the sample?

In [9]:
len(df)

7314

How many unique values are there?

In [10]:
len(df.value.unique())

1223

What are the symbols that returned values?

In [11]:
df.symbol.value_counts().sort_index()

CanvasRenderingContext2D.fillStyle                                       39  
CanvasRenderingContext2D.font                                            18  
CanvasRenderingContext2D.globalCompositeOperation                        2   
CanvasRenderingContext2D.lineWidth                                       10  
CanvasRenderingContext2D.shadowBlur                                      20  
CanvasRenderingContext2D.shadowColor                                     20  
CanvasRenderingContext2D.shadowOffsetX                                   20  
CanvasRenderingContext2D.shadowOffsetY                                   20  
CanvasRenderingContext2D.strokeStyle                                     4   
CanvasRenderingContext2D.textBaseline                                    7   
HTMLCanvasElement.height                                                 35  
HTMLCanvasElement.nodeName                                               2   
HTMLCanvasElement.nodeType                                      

The dataset contains cookies that were accessed via the JS API (`window.document.cookie`). They are stored as key-value pairs of the form `<name>=<value>`, delimited by `;`. Note, however, that this only represents a subset of all cookies used, since we don't have visibility into those that are sent with each request.

In [12]:
df_cookies = df[df.symbol == "window.document.cookie"]

In [13]:
df_cookies.head()

,location,script_url,symbol,value,value_1000,value_len
50,https://www.syracuse.edu/about/,https://www.google-analytics.com/analytics.js,window.document.cookie,"_ga=GA1.2.1687044407.1513387667; path=/; expires=Mon, 16 Dec 2019 01:27:47 GMT; domain=syracuse.edu;","_ga=GA1.2.1687044407.1513387667; path=/; expires=Mon, 16 Dec 2019 01:27:47 GMT; domain=syracuse.edu;",100
51,https://www.syracuse.edu/about/,https://www.google-analytics.com/analytics.js,window.document.cookie,_ga=GA1.2.1687044407.1513387667,_ga=GA1.2.1687044407.1513387667,31
52,https://www.syracuse.edu/about/,https://www.google-analytics.com/analytics.js,window.document.cookie,_ga=GA1.2.1687044407.1513387667,_ga=GA1.2.1687044407.1513387667,31
53,https://www.syracuse.edu/about/,https://www.google-analytics.com/analytics.js,window.document.cookie,_ga=GA1.2.1687044407.1513387667,_ga=GA1.2.1687044407.1513387667,31
54,https://www.syracuse.edu/about/,https://www.google-analytics.com/analytics.js,window.document.cookie,"_gid=GA1.2.794143.1513387667; path=/; expires=Sun, 17 Dec 2017 01:27:47 GMT; domain=syracuse.edu;","_gid=GA1.2.794143.1513387667; path=/; expires=Sun, 17 Dec 2017 01:27:47 GMT; domain=syracuse.edu;",97


How many cookie values are there in the sample?

In [14]:
len(df_cookies)

3189

How many cookies were accessed in each call, and what is the distribution?

In [15]:
def parse_cookie_entries(contents):
    """Split value contents into '<name>=<value>' strings."""
    entries = contents.split(";")
    cleaned_entries = [e.strip() for e in entries if len(e) > 0]
    return cleaned_entries

In [16]:
cookie_num_entries = df_cookies.value.apply(lambda c: len(parse_cookie_entries(c)))

In [17]:
cookie_num_entries.value_counts().sort_index()

1     158
2     308
3     363
4     681
5     180
6     174
7     154
8     48 
9     115
10    67 
11    44 
12    77 
13    58 
14    114
15    67 
16    107
17    96 
18    59 
19    61 
20    13 
21    23 
22    10 
23    13 
24    6  
25    14 
26    21 
27    22 
28    2  
29    16 
30    3  
31    5  
32    23 
33    3  
34    84 
Name: value, dtype: int64

## Working with the full dataset in Spark

We use [findspark](https://github.com/minrk/findspark) to set up spark.

In [18]:
import findspark

findspark.init()  # Load Spark, handling necessary environment settings.

from pyspark import SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.functions as fun

sc = SparkContext(appName="Overscripted")
spark = SparkSession(sc)

Download the full dataset and unzip into the data dir. It will be split across subdirs `0/`, `1/`, `2/`, `3/`. Read it into Spark, restricted to the columns of interest.

In [19]:
FULL_DATA_DIRS = ["0", "1", "2", "3"]

In [20]:
sdf = spark.read.option("mergeSchema", "true")\
    .parquet(*[os.path.join(DATA_DIR, d) for d in FULL_DATA_DIRS])

In [21]:
sdf = sdf.select(COLUMNS)

The `value` column in the full dataset has some extremely long strings that cause problems when scanning the dataset. For now, we drop this and work with the `value_1000` column instead.

In [22]:
sdf = sdf.drop("value")

In [23]:
sdf.limit(5).toPandas()

,location,script_url,symbol,value_1000,value_len
0,https://tramites.saime.gob.ve/index.php?r=verificacion/verificacion,https://tramites.saime.gob.ve/assets/e5116c7f/jquery.min.js,window.navigator.userAgent,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,68
1,https://tramites.saime.gob.ve/index.php?r=verificacion/verificacion,https://tramites.saime.gob.ve/assets/e5116c7f/jui/js/jquery-ui.min.js,window.navigator.userAgent,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,68
2,https://tramites.saime.gob.ve/index.php?r=verificacion/verificacion,https://tramites.saime.gob.ve/themes/saime/js/jquery.ui.js,window.navigator.userAgent,Mozilla/5.0 (X11; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0,68
3,https://tramites.saime.gob.ve/index.php?r=verificacion/verificacion,https://www.google-analytics.com/analytics.js,window.screen.colorDepth,24,2
4,https://tramites.saime.gob.ve/index.php?r=verificacion/verificacion,https://www.google-analytics.com/analytics.js,window.navigator.plugins[Shockwave Flash].name,Shockwave Flash,15


Restrict to operations that returned a value.

In [24]:
sdf = sdf.where("value_len > 0")

How many rows with values are there in the dataset?

In [25]:
def fmt(num):
    print(format(num, ","))

In [26]:
fmt(sdf.count())

84,786,487


What are the symbols that returned values?

In [27]:
sdf.groupBy("symbol").count()\
    .orderBy("symbol")\
    .toPandas()

,symbol,count
0,AnalyserNode.channelCount,3
1,AnalyserNode.channelCountMode,3
2,AnalyserNode.channelInterpretation,3
3,AnalyserNode.fftSize,774
4,AnalyserNode.frequencyBinCount,770
5,AnalyserNode.maxDecibels,771
6,AnalyserNode.minDecibels,769
7,AnalyserNode.numberOfInputs,3
8,AnalyserNode.numberOfOutputs,3
9,AnalyserNode.smoothingTimeConstant,770


How many of the values are numeric (integer)?

In [28]:
sdf = sdf.withColumn("value_is_numeric",
                     sdf.value_1000.cast("int").isNotNull())

In [29]:
sdf.groupBy("value_is_numeric").count().toPandas()

,value_is_numeric,count
0,True,5452728
1,False,79333759


How are these distributed across the symbols?

In [30]:
sdf.groupBy("symbol").pivot("value_is_numeric").count()\
    .fillna(0)\
    .orderBy("symbol")\
    .toPandas()

,symbol,false,true
0,AnalyserNode.channelCount,0,3
1,AnalyserNode.channelCountMode,3,0
2,AnalyserNode.channelInterpretation,3,0
3,AnalyserNode.fftSize,0,774
4,AnalyserNode.frequencyBinCount,0,770
5,AnalyserNode.maxDecibels,0,771
6,AnalyserNode.minDecibels,0,769
7,AnalyserNode.numberOfInputs,0,3
8,AnalyserNode.numberOfOutputs,0,3
9,AnalyserNode.smoothingTimeConstant,0,770
